**What is Keras Tuner?**

**Keras Tuner is a library designed to help optimize hyperparameters for machine learning models built using the Keras deep learning framework. It automates the process of finding the best set of hyperparameters, which can significantly improve the performance of models.**

**Why is Keras Tuner Popular?**

1. Ease of Use: It integrates seamlessly with Keras and TensorFlow, making it very user-friendly for those already familiar with these frameworks.
2. Flexibility: It supports various search algorithms like Random Search, Bayesian Optimization, and Hyperband, allowing users to choose the most suitable method for their needs.
3. Scalability: Keras Tuner can distribute the hyperparameter tuning process across multiple GPUs or even multiple machines, making it scalable for large datasets and complex models.
4. Customization: Users can define their custom search spaces and objectives, providing flexibility in optimizing a wide range of hyperparameters.

**Advantages of Keras Tuner**

1. Integration with Keras: Simplifies the process of hyperparameter tuning for Keras users.
2. Supports Multiple Search Algorithms: Provides flexibility in choosing different optimization strategies.
3. Visualization Tools: Comes with built-in tools to visualize the tuning process and results.
4. Distributed Tuning: Can perform distributed hyperparameter tuning on multiple devices.

**Disadvantages of Keras Tuner**

1. Limited to Keras Models: Primarily designed for Keras, making it less useful for models built with other frameworks.
2. Resource Intensive: Hyperparameter tuning can be computationally expensive, requiring significant time and resources.
3. Complexity for Beginners: While it simplifies hyperparameter tuning, beginners might still find it challenging to set up the search space and understand the results.

In [117]:
import pandas as pd
import numpy as np

In [118]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt

In [119]:
df = pd.read_csv("/content/diabetes.csv")

In [120]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [121]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [122]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

In [123]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X)

In [124]:
X.shape

(768, 8)

In [125]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [126]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((614, 8), (614,), (154, 8), (154,))

In [127]:
model = Sequential()
model.add(Dense(12,activation="relu",input_dim=X_train.shape[1]))
model.add(Dense(8,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [128]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 12)                108       
                                                                 
 dense_10 (Dense)            (None, 8)                 104       
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 221 (884.00 Byte)
Trainable params: 221 (884.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [129]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [130]:
model.fit(X_train,y_train,epochs=45,validation_split=0.2,batch_size=32)

Epoch 1/45
16/16 [==============================] - 1s 16ms/step - loss: 0.6457 - accuracy: 0.6599 - val_loss: 0.6478 - val_accuracy: 0.6829
Epoch 2/45
16/16 [==============================] - 0s 5ms/step - loss: 0.6199 - accuracy: 0.6884 - val_loss: 0.6302 - val_accuracy: 0.6748
Epoch 3/45
16/16 [==============================] - 0s 6ms/step - loss: 0.5976 - accuracy: 0.7108 - val_loss: 0.6131 - val_accuracy: 0.6748
Epoch 4/45
16/16 [==============================] - 0s 5ms/step - loss: 0.5779 - accuracy: 0.7088 - val_loss: 0.5981 - val_accuracy: 0.6667
Epoch 5/45
16/16 [==============================] - 0s 5ms/step - loss: 0.5612 - accuracy: 0.7230 - val_loss: 0.5860 - val_accuracy: 0.6829
Epoch 6/45
16/16 [==============================] - 0s 6ms/step - loss: 0.5441 - accuracy: 0.7434 - val_loss: 0.5720 - val_accuracy: 0.6829
Epoch 7/45
16/16 [==============================] - 0s 4ms/step - loss: 0.5302 - accuracy: 0.7475 - val_loss: 0.5599 - val_accuracy: 0.6748
Epoch 8/45
16/16 [=

1. how to select appropriate optimizers
2. No. of nodes in a Layer
3. How to select no.of hidden layers
3. All in one model

**Optimizers**

In [131]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(12,activation="relu",input_dim=X_train.shape[1]))
  model.add(Dense(1,activation="sigmoid"))

  optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","adadelta"])

  model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])

  return model

**Initialize the Random Search Tuner**

In [132]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5)

In [133]:
tuner.search(X_train,y_train,epochs=5,validation_split=0.2,verbose=1)

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6341463327407837

Best val_accuracy So Far: 0.7886179089546204
Total elapsed time: 00h 00m 09s


In [134]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [135]:
model = tuner.get_best_models(num_models=1)[0]

In [136]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                108       
                                                                 
 dense_1 (Dense)             (None, 1)                 13        
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [137]:
model.fit(X_train,y_train,epochs=50,validation_split=0.2,verbose=1,batch_size=32,initial_epoch=6)

Epoch 7/50
16/16 [==============================] - 1s 16ms/step - loss: 0.5678 - accuracy: 0.7332 - val_loss: 0.5298 - val_accuracy: 0.7886
Epoch 8/50
16/16 [==============================] - 0s 4ms/step - loss: 0.5604 - accuracy: 0.7352 - val_loss: 0.5251 - val_accuracy: 0.7886
Epoch 9/50
16/16 [==============================] - 0s 4ms/step - loss: 0.5543 - accuracy: 0.7434 - val_loss: 0.5204 - val_accuracy: 0.7805
Epoch 10/50
16/16 [==============================] - 0s 5ms/step - loss: 0.5484 - accuracy: 0.7475 - val_loss: 0.5160 - val_accuracy: 0.7642
Epoch 11/50
16/16 [==============================] - 0s 5ms/step - loss: 0.5432 - accuracy: 0.7495 - val_loss: 0.5122 - val_accuracy: 0.7642
Epoch 12/50
16/16 [==============================] - 0s 4ms/step - loss: 0.5385 - accuracy: 0.7495 - val_loss: 0.5086 - val_accuracy: 0.7642
Epoch 13/50
16/16 [==============================] - 0s 5ms/step - loss: 0.5342 - accuracy: 0.7475 - val_loss: 0.5058 - val_accuracy: 0.7642
Epoch 14/50
16/

#**2. Number of Neurons**

In [138]:
def build_model(hp):
  model = Sequential()
  units = hp.Int("units",min_value=8,max_value=512,step=8)
  model.add(Dense(units=units,activation="relu",input_dim=X_train.shape[1]))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer='rmsprop',loss="binary_crossentropy",metrics=["accuracy"])

  return model

tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,
                      directory="mydir",project_name="my_project")


In [139]:
tuner.search(X_train,y_train,epochs=5,validation_split=0.2,verbose=1)

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6991869807243347

Best val_accuracy So Far: 0.7642276287078857
Total elapsed time: 00h 00m 12s


In [140]:
tuner.get_best_hyperparameters()[0].values

{'units': 208}

In [141]:
model = tuner.get_best_models(num_models=1)[0]

In [142]:
model.fit(X_train,y_train,epochs=40,validation_split=0.2,verbose=1,batch_size=32,initial_epoch=6)

Epoch 7/40
16/16 [==============================] - 1s 16ms/step - loss: 0.4550 - accuracy: 0.7821 - val_loss: 0.4790 - val_accuracy: 0.7561
Epoch 8/40
16/16 [==============================] - 0s 5ms/step - loss: 0.4482 - accuracy: 0.7862 - val_loss: 0.4741 - val_accuracy: 0.7642
Epoch 9/40
16/16 [==============================] - 0s 4ms/step - loss: 0.4441 - accuracy: 0.7780 - val_loss: 0.4724 - val_accuracy: 0.7480
Epoch 10/40
16/16 [==============================] - 0s 5ms/step - loss: 0.4397 - accuracy: 0.7862 - val_loss: 0.4704 - val_accuracy: 0.7642
Epoch 11/40
16/16 [==============================] - 0s 5ms/step - loss: 0.4361 - accuracy: 0.7841 - val_loss: 0.4704 - val_accuracy: 0.7561
Epoch 12/40
16/16 [==============================] - 0s 5ms/step - loss: 0.4336 - accuracy: 0.7923 - val_loss: 0.4686 - val_accuracy: 0.7561
Epoch 13/40
16/16 [==============================] - 0s 4ms/step - loss: 0.4303 - accuracy: 0.7882 - val_loss: 0.4692 - val_accuracy: 0.7642
Epoch 14/40
16/

#**3. No of hidden layers**

In [143]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(72,activation="relu",input_dim=X_train.shape[1]))

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):

    model.add(Dense(72,activation="relu"))

  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model


In [144]:
tuner = kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,
                        directory="mydir",project_name="num_layer")

In [145]:
tuner.search(X_train,y_train,epochs=5,validation_split=0.2,verbose=1)

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7560975551605225

Best val_accuracy So Far: 0.7642276287078857
Total elapsed time: 00h 00m 15s


In [146]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [147]:
model = tuner.get_best_models(num_models=1)[0]

In [148]:
model.fit(X_train,y_train,epochs=40,validation_split=0.2,verbose=1,batch_size=32,initial_epoch=6)

Epoch 7/40
16/16 [==============================] - 1s 17ms/step - loss: 0.4647 - accuracy: 0.7637 - val_loss: 0.4830 - val_accuracy: 0.7642
Epoch 8/40
16/16 [==============================] - 0s 6ms/step - loss: 0.4404 - accuracy: 0.7923 - val_loss: 0.4808 - val_accuracy: 0.7724
Epoch 9/40
16/16 [==============================] - 0s 5ms/step - loss: 0.4372 - accuracy: 0.7923 - val_loss: 0.4820 - val_accuracy: 0.7642
Epoch 10/40
16/16 [==============================] - 0s 6ms/step - loss: 0.4155 - accuracy: 0.7882 - val_loss: 0.4992 - val_accuracy: 0.7561
Epoch 11/40
16/16 [==============================] - 0s 5ms/step - loss: 0.4121 - accuracy: 0.8126 - val_loss: 0.5378 - val_accuracy: 0.7398
Epoch 12/40
16/16 [==============================] - 0s 5ms/step - loss: 0.3950 - accuracy: 0.8269 - val_loss: 0.5217 - val_accuracy: 0.7317
Epoch 13/40
16/16 [==============================] - 0s 5ms/step - loss: 0.3855 - accuracy: 0.8167 - val_loss: 0.4840 - val_accuracy: 0.7561
Epoch 14/40
16/

**Combine All**

In [149]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):

    if counter == 0:

      model.add(
          Dense(
          hp.Int("units"+str(i),min_value=8,max_value=128,step=8),
          activation = hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"]),
          input_dim = X_train.shape[1]
          ))
      model.add(Dropout(hp.Float("dropout"+str(i),min_value=0.1,max_value=0.7,step=0.1)))
    else:

      model.add(
          Dense(
          hp.Int("units"+str(i),min_value=8,max_value=128,step=8),
          activation = hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"])
          )
      )
      model.add(Dropout(hp.Float("dropout"+str(i),min_value=0.1,max_value=0.7,step=0.1)))

    counter+=1

  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","adadelta"]),
                  loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [150]:
tuner = kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,
                        directory="mydir",project_name="final2")

Reloading Tuner from mydir/final2/tuner0.json


In [151]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test),verbose=1)

In [152]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 104,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'rmsprop',
 'units1': 120,
 'activation1': 'relu',
 'dropout1': 0.30000000000000004,
 'units2': 32,
 'activation2': 'tanh',
 'dropout2': 0.4,
 'units3': 56,
 'activation3': 'tanh',
 'dropout3': 0.6,
 'units4': 120,
 'activation4': 'relu',
 'dropout4': 0.2,
 'units5': 24,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 72,
 'activation6': 'sigmoid',
 'dropout6': 0.6,
 'units7': 112,
 'activation7': 'sigmoid',
 'dropout7': 0.2}

In [153]:
model = tuner.get_best_models(num_models=1)[0]

In [154]:
model.fit(X_train,y_train,epochs=120,initial_epoch=6,validation_split=0.2,verbose=1)

Epoch 7/120
16/16 [==============================] - 1s 25ms/step - loss: 0.5136 - accuracy: 0.7576 - val_loss: 0.4426 - val_accuracy: 0.7724
Epoch 8/120
16/16 [==============================] - 0s 9ms/step - loss: 0.4776 - accuracy: 0.7882 - val_loss: 0.4586 - val_accuracy: 0.7724
Epoch 9/120
16/16 [==============================] - 0s 8ms/step - loss: 0.4782 - accuracy: 0.7760 - val_loss: 0.4542 - val_accuracy: 0.7805
Epoch 10/120
16/16 [==============================] - 0s 8ms/step - loss: 0.4851 - accuracy: 0.7984 - val_loss: 0.4573 - val_accuracy: 0.7642
Epoch 11/120
16/16 [==============================] - 0s 9ms/step - loss: 0.4948 - accuracy: 0.7719 - val_loss: 0.4628 - val_accuracy: 0.7561
Epoch 12/120
16/16 [==============================] - 0s 9ms/step - loss: 0.4808 - accuracy: 0.7658 - val_loss: 0.4656 - val_accuracy: 0.7642
Epoch 13/120
16/16 [==============================] - 0s 9ms/step - loss: 0.4766 - accuracy: 0.7699 - val_loss: 0.4687 - val_accuracy: 0.7561
Epoch 14

**More complex and easy way**

In [155]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [156]:
#build model
def build_model(hp):
    model = Sequential()
    num_layers = hp.Int("num_layers", min_value=1, max_value=10)

    for i in range(num_layers):
        if i == 0:
            model.add(Dense(units=hp.Int("units" + str(i), min_value=4, max_value=128, step=6),
                            activation=hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
                            input_dim=X_train.shape[1]))
        else:
            model.add(Dense(units=hp.Int("units" + str(i), min_value=4, max_value=128, step=6),
                            activation=hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"])))

        model.add(Dropout(rate=hp.Float("dropout" + str(i), min_value=0.1, max_value=0.7, step=0.1)))
        model.add(BatchNormalization())

    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop', 'adadelta'])
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    else:
        optimizer = Adadelta(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [157]:
# Callbacks definition
callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2, verbose=1)
]

In [158]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    directory="my_dir",
    project_name="hyperparameter_tuning"
)

Reloading Tuner from my_dir/hyperparameter_tuning/tuner0.json


In [159]:
# Running the search with the callbacks included and setting the batch size in the search method
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test),
             callbacks=callbacks,
             batch_size=HyperParameters().Choice("batch_size", values=[12, 16, 24, 32, 64]),
             verbose=1)

In [160]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 100,
 'activation0': 'relu',
 'dropout0': 0.5,
 'optimizer': 'adadelta',
 'learning_rate': 0.0001,
 'units1': 94,
 'activation1': 'tanh',
 'dropout1': 0.1,
 'units2': 4,
 'activation2': 'sigmoid',
 'dropout2': 0.5,
 'units3': 22,
 'activation3': 'sigmoid',
 'dropout3': 0.30000000000000004,
 'units4': 34,
 'activation4': 'sigmoid',
 'dropout4': 0.2,
 'units5': 82,
 'activation5': 'sigmoid',
 'dropout5': 0.2,
 'units6': 124,
 'activation6': 'relu',
 'dropout6': 0.5,
 'units7': 46,
 'activation7': 'sigmoid',
 'dropout7': 0.2}

In [161]:
model = tuner.get_best_models(num_models=1)[0]

In [162]:
model.fit(X_train,y_train,epochs=120,initial_epoch=5,validation_data=(X_test,y_test),verbose=1)

Epoch 6/120
20/20 [==============================] - 6s 47ms/step - loss: 0.8335 - accuracy: 0.5375 - val_loss: 0.6865 - val_accuracy: 0.6429
Epoch 7/120
20/20 [==============================] - 0s 14ms/step - loss: 0.8979 - accuracy: 0.5065 - val_loss: 0.6856 - val_accuracy: 0.6429
Epoch 8/120
20/20 [==============================] - 0s 12ms/step - loss: 0.8883 - accuracy: 0.5065 - val_loss: 0.6826 - val_accuracy: 0.6429
Epoch 9/120
20/20 [==============================] - 0s 11ms/step - loss: 0.8811 - accuracy: 0.5114 - val_loss: 0.6832 - val_accuracy: 0.6429
Epoch 10/120
20/20 [==============================] - 0s 9ms/step - loss: 0.8839 - accuracy: 0.4902 - val_loss: 0.6836 - val_accuracy: 0.6429
Epoch 11/120
20/20 [==============================] - 0s 9ms/step - loss: 0.8958 - accuracy: 0.4902 - val_loss: 0.6833 - val_accuracy: 0.6429
Epoch 12/120
20/20 [==============================] - 0s 9ms/step - loss: 0.8309 - accuracy: 0.5195 - val_loss: 0.6837 - val_accuracy: 0.6429
Epoch 

**You can add more**